# MongoDB Atlas

```{=mdx}
:::tip 兼容性
仅适用于 Node.js。

您仍然可以创建使用 MongoDB 的 API 路由，只需将 `runtime` 变量设置为 `nodejs`，如下所示：

`export const runtime = "nodejs";`

您可以在 Next.js 文档中[此处](https://nextjs.org/docs/app/building-your-application/rendering/edge-and-nodejs-runtimes)阅读有关 Edge 运行时的更多信息。
:::
```

本指南提供了 MongoDB Atlas [向量存储](/docs/concepts/#vectorstores) 的快速入门概述。如需了解所有 `MongoDBAtlasVectorSearch` 功能和配置的详细文档，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_mongodb.MongoDBAtlasVectorSearch.html)。

## 概述

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/mongodb_atlas/) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`MongoDBAtlasVectorSearch`](https://api.js.langchain.com/classes/langchain_mongodb.MongoDBAtlasVectorSearch.html) | [`@langchain/mongodb`](https://www.npmjs.com/package/@langchain/mongodb) | ✅ | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/mongodb?style=flat-square&label=%20&) |

## 配置

要使用 MongoDB Atlas 向量存储，您需要配置一个 MongoDB Atlas 集群并安装 `@langchain/mongodb` 集成包。

### 初始集群配置

要创建一个 MongoDB Atlas 集群，请前往 [MongoDB Atlas 网站](https://www.mongodb.com/products/platform/atlas-database) 并在需要时创建一个账户。

创建并命名一个集群后，在 `Database` 下找到它。选择 `Browse Collections`，然后创建一个空白集合或使用提供的示例数据创建一个集合。

**注意：** 创建的集群必须是 MongoDB 7.0 或更高版本。

### 创建索引

配置好集群后，您需要在要搜索的集合字段上创建一个索引。

切换到 `Atlas Search` 选项卡并点击 `Create Search Index`。接着请确保选择 `Atlas Vector Search - JSON Editor`，然后选择适当的数据库和集合，并将以下内容粘贴到文本框中：

```json
{
  "fields": [
    {
      "numDimensions": 1536,
      "path": "embedding",
      "similarity": "euclidean",
      "type": "vector"
    }
  ]
}
```

请注意，dimensions 属性应该与您使用的嵌入向量的维度相匹配。例如，Cohere 嵌入向量有 1024 个维度，而默认情况下 OpenAI 嵌入向量有 1536 个维度：

注意：默认情况下，向量存储期望索引名称为 default，索引集合字段名称为 embedding，原始文本字段名称为 text。您应该根据您的索引名称集合模式初始化向量存储，如下面所示。

最后，继续构建索引。

### 嵌入向量

本指南还将使用 [OpenAI 嵌入向量](/docs/integrations/text_embedding/openai)，这需要您安装 `@langchain/openai` 集成包。如果您愿意，也可以使用 [其他支持的嵌入向量模型](/docs/integrations/text_embedding)。

### 安装

安装以下包：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/mongodb mongodb @langchain/openai @langchain/core
</Npm2Yarn>
```

### 凭证

完成上述步骤后，从 Mongo 仪表板中的 `Connect` 按钮设置 `MONGODB_ATLAS_URI` 环境变量。您还需要数据库名称和集合名称：

```typescript
process.env.MONGODB_ATLAS_URI = "your-atlas-url";
process.env.MONGODB_ATLAS_COLLECTION_NAME = "your-atlas-db-name";
process.env.MONGODB_ATLAS_DB_NAME = "your-atlas-db-name";
```

如果您在本指南中使用 OpenAI 嵌入向量，则还需要设置您的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果您想对模型调用进行自动追踪，也可以取消注释下面的内容并设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

按照上述方式配置好集群后，你可以按如下方式初始化你的向量存储：

In [1]:
import { MongoDBAtlasVectorSearch } from "@langchain/mongodb";
import { OpenAIEmbeddings } from "@langchain/openai";
import { MongoClient } from "mongodb";

const client = new MongoClient(process.env.MONGODB_ATLAS_URI || "");
const collection = client.db(process.env.MONGODB_ATLAS_DB_NAME)
  .collection(process.env.MONGODB_ATLAS_COLLECTION_NAME);

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const vectorStore = new MongoDBAtlasVectorSearch(embeddings, {
  collection: collection,
  indexName: "vector_index", // The name of the Atlas search index. Defaults to "default"
  textKey: "text", // The name of the collection field containing the raw content. Defaults to "text"
  embeddingKey: "embedding", // The name of the collection field containing the embedded text. Defaults to "embedding"
});

## 管理向量存储

### 向向量存储中添加项目

现在你可以将文档添加到你的向量存储中：

In [2]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents, { ids: ["1", "2", "3", "4"] });

[ '1', '2', '3', '4' ]


**注意：** 添加文档后，会有一段短暂延迟，之后这些文档才能被查询。

使用与现有文档相同的 `id` 添加文档将会更新现有文档。

### 从向量存储中删除条目

In [3]:
await vectorStore.delete({ ids: ["4"] });

## 查询向量存储

一旦创建了向量存储并添加了相关文档，您很可能希望在链或代理运行期间对其进行查询。

### 直接查询

执行一个简单的相似性搜索可以按如下方式进行：

In [5]:
const similaritySearchResults = await vectorStore.similaritySearch("biology", 2);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"_id":"1","source":"https://example.com"}]
* Mitochondria are made out of lipids [{"_id":"3","source":"https://example.com"}]


### 过滤

MongoDB Atlas 支持在其他字段上对结果进行预过滤。你需要通过更新最初创建的索引来定义计划过滤的元数据字段。以下是一个示例：

```json
{
  "fields": [
    {
      "numDimensions": 1024,
      "path": "embedding",
      "similarity": "euclidean",
      "type": "vector"
    },
    {
      "path": "source",
      "type": "filter"
    }
  ]
}
```

上面示例中，`fields` 中的第一个条目是向量索引，第二个条目是你希望进行过滤的元数据属性。属性的名称是 `path` 键的值。因此，上面的索引允许我们对名为 `source` 的元数据字段进行搜索。

然后，在你的代码中，你可以使用 [MQL 查询操作符](https://www.mongodb.com/docs/manual/reference/operator/query/) 进行过滤。

以下示例说明了这一点：

In [9]:
const filter = {
  preFilter: {
    source: {
      $eq: "https://example.com",
    },
  },
}

const filteredResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of filteredResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"_id":"1","source":"https://example.com"}]
* Mitochondria are made out of lipids [{"_id":"3","source":"https://example.com"}]


### 返回分数

如果你想执行相似性搜索并获得相应的分数，可以运行：

In [10]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.374] The powerhouse of the cell is the mitochondria [{"_id":"1","source":"https://example.com"}]
* [SIM=0.370] Mitochondria are made out of lipids [{"_id":"3","source":"https://example.com"}]


### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在你的链中更方便地使用。

In [11]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { _id: '1', source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { _id: '3', source: 'https://example.com' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [操作指南：使用 RAG 进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## 关闭连接

完成操作后，请确保关闭客户端实例，以避免过度消耗资源：

In [12]:
await client.close();

## API 参考文档

如需详细了解所有 `MongoDBAtlasVectorSearch` 功能和配置，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_mongodb.MongoDBAtlasVectorSearch.html)。